# Avatar Renderer MCP - Demo Notebook

**Transform static images into dynamic AI-powered avatars with realistic expressions and voice synchronization**

This notebook demonstrates how to:
1. Install and set up Avatar Renderer MCP
2. Generate your first "Hello World" talking avatar
3. Create professional demo videos for marketing
4. Use different quality modes (real-time vs high-quality)

---

**Website:** [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

**Repository:** [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)

## 1. Installation & Setup

First, let's install the package and its dependencies.

In [ ]:
# Install uv package manager (if not already installed)
!pip install -q uv

In [ ]:
# Install avatar-renderer-mcp and dependencies
# Note: This assumes you're in the repository root directory
!uv pip install -e .

In [ ]:
# Install external dependencies (SadTalker, Wav2Lip, etc.)
# This may take a few minutes
!make install-git-deps

## 2. Imports & Configuration

In [ ]:
import os
import sys
from pathlib import Path
from IPython.display import Video, HTML, display
import warnings

warnings.filterwarnings('ignore')

# Add external dependencies to Python path
ext_deps = Path.cwd() / "external_deps"
if ext_deps.exists():
    sys.path.insert(0, str(ext_deps))
    print(f"✓ Added external dependencies: {ext_deps}")

# Import the pipeline
from app.pipeline import render_pipeline

print("✓ All imports successful!")

In [ ]:
# Check GPU availability
import torch

cuda_available = torch.cuda.is_available()
print(f"GPU Available: {cuda_available}")
if cuda_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Using CPU mode (slower but works).")

## 3. Setup Directories & Assets

In [ ]:
# Create output directory for generated videos
output_dir = Path("demo_outputs")
output_dir.mkdir(exist_ok=True)
print(f"✓ Output directory: {output_dir.absolute()}")

# Check for test assets
test_image = Path("tests/assets/alice.png")
test_audio = Path("tests/assets/hello.wav")

if test_image.exists() and test_audio.exists():
    print(f"✓ Found test assets:")
    print(f"  - Image: {test_image}")
    print(f"  - Audio: {test_audio}")
else:
    print("⚠️  Test assets not found. You'll need to provide your own image and audio files.")

## 4. Hello World - Your First Avatar!

Let's create a simple "Hello World" talking avatar. This uses the test assets included in the repository.

In [ ]:
# Generate Hello World video
output_video = output_dir / "hello_world.mp4"

print("🎬 Generating Hello World avatar...")
print("This may take 30-60 seconds depending on your hardware...\n")

try:
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(output_video),
        quality_mode="auto"  # Automatically selects best mode for your hardware
    )
    print(f"\n✅ Success! Video saved to: {result}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure models are downloaded: make download-models")
    print("2. Check GPU availability if using high-quality mode")
    print("3. Try quality_mode='real_time' for CPU-only systems")

In [ ]:
# Display the generated video
if output_video.exists():
    print("🎥 Your first AI avatar:")
    display(Video(str(output_video), width=640, embed=True))
else:
    print("Video file not found. Check the error messages above.")

## 5. Create Custom Demo Videos

Now let's create professional demo videos for your website. You can use your own images and audio files.

In [ ]:
# Helper function to display videos inline
def display_video(video_path, title="Generated Video", width=640):
    """Display video with title"""
    if Path(video_path).exists():
        print(f"\n{'='*60}")
        print(f"🎬 {title}")
        print(f"{'='*60}")
        display(Video(str(video_path), width=width, embed=True))
        file_size = Path(video_path).stat().st_size / (1024 * 1024)  # MB
        print(f"\n📊 File size: {file_size:.2f} MB")
        print(f"📁 Location: {video_path}")
    else:
        print(f"❌ Video not found: {video_path}")

### Example 1: Professional Avatar (Customer Support)

Perfect for:
- Customer support representatives
- Company announcements
- Professional presentations

In [ ]:
# Example: Create a professional customer support avatar
# Replace these paths with your own assets

professional_video = output_dir / "professional_demo.mp4"

print("🎬 Creating professional customer support avatar...")

# Using the test assets as an example
# In production, you'd use: your_image.png and your_script.wav
try:
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(professional_video),
        quality_mode="high_quality"  # Best quality for professional content
    )
    display_video(result, "Professional Customer Support Avatar")
except Exception as e:
    print(f"⚠️  High-quality mode failed: {e}")
    print("Trying real-time mode instead...")
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(professional_video),
        quality_mode="real_time"
    )
    display_video(result, "Professional Customer Support Avatar (Real-time Mode)")

### Example 2: Real-Time Avatar (Live Streaming)

Perfect for:
- Live news broadcasts
- Real-time chatbots
- Interactive virtual assistants

**Target: <3s latency**

In [ ]:
import time

realtime_video = output_dir / "realtime_demo.mp4"

print("⚡ Creating real-time avatar (optimized for speed)...")

start_time = time.time()

result = render_pipeline(
    face_image=str(test_image),
    audio=str(test_audio),
    out_path=str(realtime_video),
    quality_mode="real_time"  # Fast processing for streaming
)

processing_time = time.time() - start_time

print(f"\n⏱️  Processing time: {processing_time:.2f} seconds")
display_video(result, "Real-Time Avatar (Live Streaming Ready)")

## 6. Quality Mode Comparison

Let's compare the two quality modes:

| Feature | Real-Time | High-Quality |
|---------|-----------|-------------|
| **Speed** | <3s | ~10-30s |
| **GPU Required** | No | Yes |
| **Bitrate** | 2 Mbps | 6 Mbps |
| **Enhancement** | None | GFPGAN |
| **Best For** | Live streaming | Pre-recorded content |
| **Pipeline** | SadTalker + Wav2Lip | FOMM + Diff2Lip + GFPGAN |

## 7. Create Your Own Content

Ready to create your own avatar videos? Here's a template:

In [ ]:
# Template for creating your own avatar videos

def create_custom_avatar(
    image_path: str,
    audio_path: str,
    output_name: str,
    quality: str = "auto",
    description: str = "Custom Avatar"
):
    """
    Create a custom avatar video
    
    Args:
        image_path: Path to avatar image (PNG/JPG)
        audio_path: Path to audio file (WAV/MP3)
        output_name: Name for output video (without extension)
        quality: 'real_time', 'high_quality', or 'auto'
        description: Description for display
    """
    output_path = output_dir / f"{output_name}.mp4"
    
    print(f"🎬 Creating {description}...")
    print(f"   Image: {image_path}")
    print(f"   Audio: {audio_path}")
    print(f"   Quality: {quality}\n")
    
    try:
        result = render_pipeline(
            face_image=image_path,
            audio=audio_path,
            out_path=str(output_path),
            quality_mode=quality
        )
        display_video(result, description)
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("✓ Custom avatar function ready!")
print("\nUsage example:")
print('create_custom_avatar(')
print('    image_path="path/to/your/image.png",')
print('    audio_path="path/to/your/audio.wav",')
print('    output_name="my_avatar",')
print('    quality="high_quality",')
print('    description="My Custom Avatar"')
print(')')

In [ ]:
# Example: Create your own avatar
# Uncomment and modify the paths below:

# create_custom_avatar(
#     image_path="path/to/your/image.png",
#     audio_path="path/to/your/audio.wav",
#     output_name="my_first_avatar",
#     quality="auto",
#     description="My First Custom Avatar"
# )

## 8. Tips for Creating Great Demo Videos

### 🖼️ Image Requirements
- **Format**: PNG or JPG
- **Face**: Clear, front-facing portrait
- **Resolution**: Minimum 512x512, recommended 1024x1024
- **Lighting**: Well-lit, avoid harsh shadows
- **Background**: Clean, uncluttered

### 🎤 Audio Requirements
- **Format**: WAV or MP3
- **Sample Rate**: 16kHz recommended
- **Quality**: Clear speech, minimal background noise
- **Length**: Keep it concise (10-30 seconds for demos)

### 🎬 Demo Video Best Practices
1. **Keep it short**: 5-15 seconds for website demos
2. **Clear message**: Focus on one key benefit
3. **Professional appearance**: Use high-quality mode for marketing
4. **Multiple variations**: Create videos for different use cases
5. **Test on different devices**: Ensure compatibility

### 📊 Recommended Demo Topics
1. **Customer Support**: "Hello! I'm here to help you 24/7"
2. **AI Teacher**: "Welcome to today's lesson on..."
3. **Brand Ambassador**: "Discover our latest product..."
4. **Game NPC**: "Greetings, traveler! What brings you here?"
5. **News Anchor**: "Good evening, here are today's top stories..."

## 9. Batch Processing (Multiple Videos)

Create multiple demo videos at once for your website:

In [ ]:
def batch_create_demos(demos_config):
    """
    Create multiple demo videos from a configuration list
    
    Args:
        demos_config: List of dictionaries with demo configurations
    """
    results = []
    
    print(f"🎬 Starting batch creation of {len(demos_config)} videos...\n")
    
    for i, config in enumerate(demos_config, 1):
        print(f"\n{'='*60}")
        print(f"Processing {i}/{len(demos_config)}: {config['name']}")
        print(f"{'='*60}")
        
        result = create_custom_avatar(
            image_path=config['image'],
            audio_path=config['audio'],
            output_name=config['output'],
            quality=config.get('quality', 'auto'),
            description=config['name']
        )
        
        results.append({
            'name': config['name'],
            'output': result,
            'success': result is not None
        })
    
    # Summary
    print(f"\n\n{'='*60}")
    print("📊 BATCH PROCESSING SUMMARY")
    print(f"{'='*60}")
    
    successful = sum(1 for r in results if r['success'])
    print(f"✅ Successful: {successful}/{len(results)}")
    
    if successful < len(results):
        print(f"❌ Failed: {len(results) - successful}")
        print("\nFailed videos:")
        for r in results:
            if not r['success']:
                print(f"  - {r['name']}")
    
    return results

# Example configuration for multiple demos
demo_configs = [
    {
        'name': 'Hello World Demo',
        'image': str(test_image),
        'audio': str(test_audio),
        'output': 'demo_1_hello',
        'quality': 'auto'
    },
    # Add more demos here:
    # {
    #     'name': 'Customer Support Demo',
    #     'image': 'assets/support_avatar.png',
    #     'audio': 'assets/support_script.wav',
    #     'output': 'demo_2_support',
    #     'quality': 'high_quality'
    # },
]

# Uncomment to run batch processing:
# results = batch_create_demos(demo_configs)

## 10. API Reference

### Core Function: `render_pipeline()`

```python
render_pipeline(
    face_image: str,           # Path to avatar image (PNG/JPG)
    audio: str,                # Path to audio file (WAV/MP3)
    out_path: str,             # Output video path
    reference_video: str = None,  # Optional driving video
    quality_mode: str = "auto"    # "real_time", "high_quality", or "auto"
) -> str  # Returns absolute path to generated video
```

### Quality Modes

#### `real_time`
- **Speed**: <3 seconds latency
- **Pipeline**: SadTalker + Wav2Lip
- **GPU**: Optional (CPU fallback available)
- **Use cases**: Live streaming, chatbots, real-time interactions

#### `high_quality`
- **Quality**: Maximum with GFPGAN enhancement
- **Pipeline**: FOMM + Diff2Lip + GFPGAN
- **GPU**: Required (V100 or better recommended)
- **Use cases**: YouTube content, marketing videos, professional productions

#### `auto` (default)
- Automatically selects the best mode based on:
  - GPU availability
  - Model checkpoint availability
  - System resources

## 11. Next Steps

### 📚 Learn More
- Read the full documentation: [README.md](README.md)
- Explore quality modes: [docs/QUALITY_MODES.md](docs/QUALITY_MODES.md)
- Check out the API: Start the FastAPI server with `make run`

### 🚀 Deploy to Production
1. **Docker**: `make docker-build && make docker-run`
2. **Kubernetes**: `helm install avatar-renderer ./charts/avatar-renderer`
3. **Frontend**: Deploy the Next.js app from `/frontend` to Vercel

### 🎨 Create More Content
1. Prepare your avatar images and scripts
2. Use this notebook to generate demo videos
3. Upload to your website at [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

### 💡 Advanced Features
- **MCP Integration**: Use the STDIO server for AI agent communication
- **REST API**: Integrate with your applications via FastAPI
- **Custom Models**: Fine-tune models for your specific use case
- **Batch Processing**: Process multiple videos efficiently

---

## 🎉 Congratulations!

You've successfully created your first AI-powered talking avatar!

**Need help?** Check out:
- 🌐 Website: [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)
- 📦 Repository: [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)
- 📧 Contact: contact@ruslanmv.com
- 🌟 Author: [Ruslan Magana Vsevolodovna](https://ruslanmv.com)

---

*Transform static images into dynamic, AI-powered avatars with realistic expressions and voice synchronization*